# Retrieval Augmented Generation (RAG) with Azure OpenAI and Azure AI Search

In this notebook we will showcase how to implement a Retrieval Augement Generation pipeline using Azure AI Search.

## Environment setup
Before executing the following cells, make sure to set the following environment variables in the `.env` file or export them:
* `AZURE_OPENAI_KEY`
* `AZURE_OPENAI_ENDPOINT`
* `MODEL_DEPLOYMENT_NAME`
* `EMBEDDING_DEPLOYMENT_NAME`
* `VECTOR_STORE_ADDRESS`
* `VECTOR_STORE_PASSWORD`
* `INDEX_NAME`

<br/>
<img src="../assets/keys_endpoint.png" width="800"/>

In [1]:
!pip install -q langchain==0.1.16 python-dotenv langchain_openai azure-search-documents azure-identity


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
import os

from langchain.vectorstores import AzureSearch
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Create Index

In [7]:
openai_api_version = "2024-02-01"

embeddings = AzureOpenAIEmbeddings(
    deployment=os.getenv('EMBEDDING_DEPLOYMENT_NAME'),
    openai_api_version=openai_api_version,
)

In [8]:
from azure.search.documents.indexes.models import (
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
)

embedding_function = embeddings.embed_query

fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(embedding_function("Text")),
        vector_search_profile_name="myHnswProfile",
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
]

## Create the vector store on Azure

In [9]:
index_name: str = "ai-academy"

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.getenv('VECTOR_STORE_ADDRESS'),
    azure_search_key=os.getenv('VECTOR_STORE_PASSWORD'),
    index_name=index_name,
    embedding_function=embedding_function,
    fields=fields,
    # needed for semantic ranking
    semantic_configuration_name = 'my-config',
)

## Populate the vectorstore

For this notebook, we've scraped langchain's blogs (see `data/`).

In [10]:
text_loader_kwargs={'autodetect_encoding': True}

loader = DirectoryLoader(
    '../data/langchain_blog_posts/',
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs=text_loader_kwargs,
    #silent_errors=True
)

documents = loader.load()

#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)
docs = text_splitter.split_documents(documents)

len(docs)

429

In [11]:
vector_store.add_documents(documents=docs)

['...']

## Similarity Searches

With Azure AI Search, we can now perform similarity searches on the documents we've indexed.

### Simple Similarity search

In [12]:
docs = vector_store.search(
    query="What is langchain?",
    k=3,
    search_type="similarity",
)

for doc in docs:
    print(doc.page_content)
    print('-' * 50)

"LangChain's (the company's) goal is to make it as easy as possible to develop LLM applications"

said Harrison Chase, co-founder and CEO of LangChain.

"To that end, we realized pretty early that what was needed - and missing - wasn't just an open source tool like LangChain, but also a complementary platform for managing these new types of applications. To that end, we built LangSmith - which is usable with or without LangChain and let's users easily debug, monitor, test, evaluate, and now (with the recently launched Hub) share and collaborate on their LLM applications.”



What Are LangSmith Traces?
--------------------------------------------------
LangChain Expression Language creates chains that integrate seamlessly with LangSmith. Here is a trace for the above:

You can inspect the trace here. Previously, when creating a custom chain there was actually a good bit of work to be done to make sure callbacks were passed through correctly so that it could be traced correctly. With Lan

### Similarity Search with scores

In [13]:
docs = vector_store.similarity_search_with_relevance_scores(
    query="How to interact with SQL using langchain?",
    k=3,
    score_threshold=.9,
)

for doc in docs:
    print(doc)
    print('-' * 50)

(Document(page_content='The LangChain library has multiple SQL chains and even an SQL agent aimed at making interacting with data stored in SQL as easy as possible. Here are some relevant links:\n\nIntroduction\n\nMost of an enterprise’s data is traditionally stored in SQL databases. With the amount of valuable data stored there, business intelligence (BI) tools that make it easy to query and understand the data present there have risen in popularity. But what if you could just interact with a SQL database in natural language? With LLMs today, that is possible. LLMs have an understanding of SQL and are able to write it pretty well. However, there are several issues that make this a non-trivial task.\n\nThe Problems\n\nSo LLMs can write SQL - what more is needed?\n\nUnfortunately, a few things.', metadata={'source': '..\\data\\langchain_blog_posts\\blog.langchain.dev_llms-and-sql_.txt'}), 0.9001867)
--------------------------------------------------


### Hybrid Search

In a hybrid search, we can combine the similarity search with a keyword search. Vector and non-vector text fields are queried in parallel, results are merged, and top matches of the unified result set are returned.

In [14]:
docs = vector_store.hybrid_search(
    query="How to interact with SQL using langchain?",
    k=3,
)

for doc in docs:
    print(doc.page_content)
    print('-' * 50)

The LangChain library provides different tools to interact with SQL databases which can be used to build and run queries based on natural language inputs. For example, the standard SQL Toolkit draws from standard best practices that have been extensively covered in this blogpost. However, there is still room for improvement when it comes to building a custom solution and adjusting the generic tools to the specific use case. The advantage of having a plug and play toolkit contrasts with having a solution that is not flexible enough for the user to incorporate their domain-specific knowledge about the databases.
--------------------------------------------------
The LangChain library has multiple SQL chains and even an SQL agent aimed at making interacting with data stored in SQL as easy as possible. Here are some relevant links:

Introduction

Most of an enterprise’s data is traditionally stored in SQL databases. With the amount of valuable data stored there, business intelligence (BI) 

In [15]:
docs = vector_store.semantic_hybrid_search(
    query="How to interact with SQL using langchain?",
    k=3,
)

for doc in docs:
    print(doc.page_content)
    print('-' * 50)

The LangChain library provides different tools to interact with SQL databases which can be used to build and run queries based on natural language inputs. For example, the standard SQL Toolkit draws from standard best practices that have been extensively covered in this blogpost. However, there is still room for improvement when it comes to building a custom solution and adjusting the generic tools to the specific use case. The advantage of having a plug and play toolkit contrasts with having a solution that is not flexible enough for the user to incorporate their domain-specific knowledge about the databases.
--------------------------------------------------
The LangChain library has multiple SQL chains and even an SQL agent aimed at making interacting with data stored in SQL as easy as possible. Here are some relevant links:

Introduction

Most of an enterprise’s data is traditionally stored in SQL databases. With the amount of valuable data stored there, business intelligence (BI) 

## Build a QA Chain

In [32]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import PromptTemplate

llm = AzureChatOpenAI(
    deployment_name=os.getenv('MODEL_DEPLOYMENT_NAME'),
    openai_api_version=openai_api_version,
    temperature=0.,
    max_tokens=1024
)

vector_store.search_type = "similarity"
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

template = """Use the following pieces of context to answer the question at the end.
Use three sentences maximum and keep the answer as concise as possible.
Don't try to make up the answer, only use the context to answer the question.
The pieces of context refer to langchain.

Context:
{context}

Question: {question}
Helpful Answer:"""

prompt = PromptTemplate.from_template(template)

qa_chain_base = (
    RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    )
    | prompt
    | llm
    | StrOutputParser()
)

question = "Explain how to connect langchain to sql. Show me the code to do that"
print(qa_chain_base.invoke(question))

The LangChain library provides tools to interact with SQL databases, including SQL chains and an SQL agent. To connect LangChain to SQL, you can use the LangChain Expression Language, which allows you to compose chains and provides streaming, batch, and async support. Here is an example of how to connect LangChain to SQL:

```
from langchain import SQLAgent

# Create an instance of the SQLAgent
agent = SQLAgent()

# Connect to the SQL database
agent.connect(database='your_database', username='your_username', password='your_password')

# Execute a SQL query
result = agent.execute_query('SELECT * FROM your_table')

# Print the result
print(result)
```

Please note that this code is a simplified example and you may need to adjust it based on your specific use case and database configuration.


## Build a QA Chain with Reranking

In [33]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import PromptTemplate

llm = AzureChatOpenAI(
    deployment_name=os.getenv('MODEL_DEPLOYMENT_NAME'),
    openai_api_version=openai_api_version,
    temperature=0.,
    max_tokens=1024
)

vector_store.search_type = "semantic_hybrid"
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

template = """Use the following pieces of context to answer the question at the end.
Use three sentences maximum and keep the answer as concise as possible.
Don't try to make up the answer, only use the context to answer the question.
The pieces of context refer to langchain.

Context:
{context}

Question: {question}
Helpful Answer:"""

prompt = PromptTemplate.from_template(template)

qa_chain_base = (
    RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    )
    | prompt
    | llm
    | StrOutputParser()
)

question = "Explain how to connect langchain to sql. Show me the code to do that"
print(qa_chain_base.invoke(question))

To connect LangChain to SQL, you can use the provided code snippet:

```
from langchain.agents import AgentExecutor, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.sql_database import SQLDatabase

def create_agent(db_uri, agent_type=AgentType.OPENAI_FUNCTIONS, verbose=VERBOSE_LANGCHAIN, temperature=0, model="gpt-3.5-turbo-0613"):
    db = SQLDatabase.from_uri(db_uri)
    toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=temperature))
    return create_sql_agent(
        llm=ChatOpenAI(temperature=temperature, model=model),
        toolkit=toolkit,
        verbose=verbose,
        agent_type=agent_type,
    )
```

This code initializes the LangChain Agent and connects it to your SQL database. Make sure to use your OpenAI key for this, which should be kept private.
